0. Import bibliotek

In [20]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, TIMESTAMP, Float, Boolean, BigInteger, Date
import psycopg2
import sys
import uuid
from psycopg2 import OperationalError, errorcodes, errors


1. Stworzenie bazy danych

In [14]:
connection_parameters = { 
    'host': 'localhost',
    'port': 5432,
    'user' : 'postgres',
    'password' : 'postgres'
    }

In [9]:
def connect(conn_params_dic):
    conn = None
    try:
        print('Łączenie do PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Połączono pomyślnie..................")
        
    except OperationalError as err:
        print("Error: {}".format(err))
        conn = None
    
    return conn

In [10]:
conn = connect(connection_parameters)
conn.autocommit = True

if conn!=None:
    
    try:
        cursor = conn.cursor()
        # usuwanie bazy danych jeśli istnieje
        cursor.execute("DROP DATABASE IF EXISTS movierecommenderdb;")
    
        # tworzenie bazy danych
        cursor.execute("CREATE DATABASE movierecommenderdb;")
        print("Baza danych movierecommenderdb została utworzona..................")
    
        # zamknięcie połączenia
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        print("Error: {}".format(err))
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
Error: BŁĄD:  baza danych "movierecommenderdb" jest używana przez innych użytkowników
DETAIL:  3 inne sesje używają bazy danych.



1. Wczytanie danych

In [15]:
path = "C:\\Users\\aldabrow\\Desktop\\PowerBI Excercises\\INZ\\data\\transfromed_data\\"

In [16]:
df_keywords = pd.read_csv(path + "keywords.csv")
df_keywords.head()

,movieId,keywords
0,862,"jealousy, toy, boy, friendship, friends, rival..."
1,8844,"board game, disappearance, based on children's..."
2,949,"robbery, detective, bank, obsession, chase, sh..."
3,710,"cuba, falsely accused, secret identity, comput..."
4,1408,"exotic island, treasure, map, ship, scalp, pirate"


In [5]:
df_credits = pd.read_csv(path + "credits.csv")
df_credits.head()

,cast,director,movieId
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,862
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,8844
2,"Al Pacino, Robert De Niro, Val Kilmer, Jon Voi...",Michael Mann,949
3,"Pierce Brosnan, Sean Bean, Izabella Scorupco, ...",Martin Campbell,710
4,"Geena Davis, Matthew Modine, Frank Langella, M...",Renny Harlin,1408


In [17]:
df_movies = pd.read_csv(path + "movies_metadata.csv")
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,poster_path,IMBD_poster_path,IMBD_backdrop_path
0,False,Toy Story Collection,30000000,"Animation, Comedy, Family",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,English,Released,NaN,Toy Story,False,7.7,5415.0,https://image.tmdb.org/t/p/original/uXDfjJbdP4...,https://image.tmdb.org/t/p/w500//7G9915LfUQ2lV...,https://image.tmdb.org/t/p/w500//9FBwqcd9IRruE...
1,False,NaN,65000000,"Adventure, Fantasy, Family",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,https://image.tmdb.org/t/p/original/nLXYV4WmYU...,NaN,NaN
2,False,NaN,60000000,"Action, Crime, Drama, Thriller",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,"English, Español",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,https://image.tmdb.org/t/p/original/umSVjVdbVw...,NaN,NaN
3,False,James Bond Collection,58000000,"Adventure, Action, Thriller",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,"English, Pусский, Español",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,https://image.tmdb.org/t/p/original/z0ljRnNxIO...,https://image.tmdb.org/t/p/w500//HORpg5CSkmeQl...,https://image.tmdb.org/t/p/w500//6VcVl48kNKvdX...
4,False,NaN,98000000,"Action, Adventure",NaN,1408,tt0112760,en,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",...,"English, Latin",Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,False,5.7,137.0,https://image.tmdb.org/t/p/original/8g4sF1Qthl...,NaN,NaN


In [27]:
#ratings
df_ratings = pd.read_csv(path + "ratings.csv")
df_ratings.head()

,userId,movieId,rating,timestamp,normalized_rating
0,24,2,3.0,979974023,0.555556
1,24,6,4.0,979870499,0.777778
2,24,16,3.0,979870379,0.555556
3,24,17,3.0,979974163,0.555556
4,24,21,4.0,979870629,0.777778


In [28]:
# Dodaj nową kolumnę 'ratingID' i przypisz jej unikalne identyfikatory
df_ratings['ratingID'] = range(1, len(df_ratings) + 1)
df_ratings.head()

,userId,movieId,rating,timestamp,normalized_rating,ratingID
0,24,2,3.0,979974023,0.555556,1
1,24,6,4.0,979870499,0.777778,2
2,24,16,3.0,979870379,0.555556,3
3,24,17,3.0,979974163,0.555556,4
4,24,21,4.0,979870629,0.777778,5


3. Stworzenie tabel w bazie danych

In [29]:
host= "localhost"
database="movierecommenderdb"
user= connection_parameters['user']
password= connection_parameters['password']
port= "5432"

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

df_credits.to_sql('Credits', engine,if_exists='replace', index=False)
df_keywords.to_sql('Keywords', engine,if_exists='replace', index=False)
df_movies.to_sql('Movies', engine,if_exists='replace', index=False)
df_ratings.to_sql('Ratings', engine,if_exists='replace', index=False)

print("Dane zostały zapisane do bazy PostgreSQL..................")

Dane zostały zapisane do bazy PostgreSQL..................
